In [1]:
conda install pytorch torchvision cudatoolkit=10.0 -c pytorch-nightly

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [2]:
from IPython.display import clear_output 
import torch
import cv2

def offline_idnetify_items_in_video(pathToVideo,confidence,onlyhumans=True):

  #load model
  model = torch.hub.load('ultralytics/yolov5', 'custom', path='C:\\Users\\Admin\\Downloads\\last.pt')
  #model = torch.hub.load('ultralytics/yolov5', 'yolov5s')
    
  try:
  #capture video
    cap= cv2.VideoCapture(pathToVideo)
  except:
    print("path not found")
  i=0 
  #loop through each frame of the video
  while(cap.isOpened()):
    i+=1
    #read each frame in the video and store it in variable frame
    ret, frame = cap.read()
    #if video is finished exit the loop
    if ret == False:
      break
    # Inference
    results = model(frame)
    personcounter=0
    #variable name array stores the name of each object identifed in the order 
    #it was identefied in 
    nameArray=results.pandas().xyxy[0]['name']
    #variable j is used to help keep track of what object we are currently at in
    # order to access its name
    j=0
    #loop through each item found in the frame
    for item in results.xyxy[0]:
       #if item was identfied with confidnce bigger than ${minConfidince} do 2 things:
      if item[4]>confidence:
        if nameArray[j]=='person' or onlyhumans==False:
          #1 draw a square around the item
          cv2.rectangle(frame, pt1=(int(item[0]),int(item[1])), pt2=(int(item[2]),int(item[3])), color=(0,255,0), thickness=2)
          #2 draw the name of the item next to the square
          cv2.putText(frame,nameArray[j] , (int(item[2]),int(item[3])),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA)
        if nameArray[j]=='person':
          personcounter+=1
      #increase counter 
      j+=1
    
    cv2.putText(frame,"person count:"+str(personcounter) , (0,25),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),1,cv2.LINE_AA)
    
    #remove the previous frame
    #clear_output(wait=True)
    #show the current frame after being proccesesd
    cv2.imshow("image",frame)
    if cv2.waitKey(25) & 0xFF == ord('q'):
      break
    
  cap.release()
  cv2.destroyAllWindows()



In [3]:
offline_idnetify_items_in_video('C:\\Users\\Admin\\Downloads\\video_example1.mp4',0)

Using cache found in C:\Users\Admin/.cache\torch\hub\ultralytics_yolov5_master


requirements: pandas>=1.1.4 not found and is required by YOLOv5, attempting auto-update...
requirements: Command 'pip install 'pandas>=1.1.4'' returned non-zero exit status 1.
requirements: seaborn>=0.11.0 not found and is required by YOLOv5, attempting auto-update...


YOLOv5  2022-1-1 torch 1.10.1 CPU

Fusing layers... 


requirements: Command 'pip install 'seaborn>=0.11.0'' returned non-zero exit status 1.


Model Summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 
C:\Users\Admin\anaconda3\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
C:\Users\Admin\anaconda3\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
C:\Users\Admin\anaconda3\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
C:\Users\Admin\anaconda3\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  w

...

C:\Users\Admin\anaconda3\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
C:\Users\Admin\anaconda3\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
C:\Users\Admin\anaconda3\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
C:\Users\Admin\anaconda3\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. 

C:\Users\Admin\anaconda3\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
C:\Users\Admin\anaconda3\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
C:\Users\Admin\anaconda3\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
C:\Users\Admin\anaconda3\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. 

C:\Users\Admin\anaconda3\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
C:\Users\Admin\anaconda3\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
C:\Users\Admin\anaconda3\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
C:\Users\Admin\anaconda3\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. 

C:\Users\Admin\anaconda3\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
C:\Users\Admin\anaconda3\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
C:\Users\Admin\anaconda3\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
C:\Users\Admin\anaconda3\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. 

KeyboardInterrupt: 

In [2]:

import matplotlib.pyplot as plt
import cv2
import numpy as np
import torch
from IPython.display import display, Image
import ipywidgets as widgets
import threading

def oflline_detectLive(confidence = 0.4,onlyhumans=True):
    
    model = torch.hub.load('ultralytics/yolov5' ,'custom',"C:\\Users\\Admin\\Downloads\\last.pt")
    #model = torch.hub.load('ultralytics/yolov5', 'yolov5s')
    cap = cv2.VideoCapture(0)
    cap.set(3,640) # adjust width
    cap.set(4,480) # adjust height
    
    personcounter=0
    
    while True:
        success, img = cap.read()
        results = model(img)
        
        nameArray=results.pandas().xyxy[0]['name']
            #variable j is used to help keep track of what object we are currently at in
            # order to access its name
        j=0
            #loop through each item found in the frame
        for item in results.xyxy[0]:
              #if item was identfied with confidnce bigger than ${minConfidince} do 2 things:
            if item[4]>confidence :
                if nameArray[j]=='person' or onlyhumans==False:
                    #1 draw a square arpund the item
                    cv2.rectangle(img, pt1=(int(item[0]),int(item[1])), pt2 = (int(item[2]),int(item[3]) ),thickness=3, color=(0,255,0))
                    #2 draw the name of the item next to the square
                    cv2.putText(img,nameArray[j] , (int(item[0]),int(item[1])),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
                if nameArray[j]=='person':
                      personcounter+=1
              #increase counter    
            j+=1

        cv2.putText(img,"person count:"+str(personcounter) , (0,25),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),1,cv2.LINE_AA)
        cv2.imshow("Webcam", img) # This will open an independent window
        personcounter=0
        if cv2.waitKey(1) & 0xFF==ord('q'): # quit when 'q' is pressed
            cap.release()
            break

    cv2.destroyAllWindows() 
    cv2.waitKey(1)

In [3]:
oflline_detectLive()

Using cache found in C:\Users\Admin/.cache\torch\hub\ultralytics_yolov5_master


requirements: pandas>=1.1.4 not found and is required by YOLOv5, attempting auto-update...
requirements: Command 'pip install 'pandas>=1.1.4'' returned non-zero exit status 1.
requirements: seaborn>=0.11.0 not found and is required by YOLOv5, attempting auto-update...


YOLOv5  2022-1-1 torch 1.10.1 CPU

Fusing layers... 


requirements: Command 'pip install 'seaborn>=0.11.0'' returned non-zero exit status 1.


Model Summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 
C:\Users\Admin\anaconda3\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
C:\Users\Admin\anaconda3\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
C:\Users\Admin\anaconda3\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
C:\Users\Admin\anaconda3\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  w

C:\Users\Admin\anaconda3\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
C:\Users\Admin\anaconda3\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
C:\Users\Admin\anaconda3\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
C:\Users\Admin\anaconda3\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. 

C:\Users\Admin\anaconda3\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
C:\Users\Admin\anaconda3\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
C:\Users\Admin\anaconda3\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
C:\Users\Admin\anaconda3\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. 

KeyboardInterrupt: 